In [2]:
import pandas as pd
# import lightgbm as lgb
# import xgboost as xgb
# from catboost import CatBoostClassifier, Pool
from metrics import  accuracy
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, SelectFromModel, RFE, VarianceThreshold, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier

from sklearn.feature_extraction.text import CountVectorizer
from experiments import *

# from catboost import CatBoostClassifier, Pool
import matplotlib.pyplot as plt

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning

simplefilter("ignore", category=ConvergenceWarning)
simplefilter("ignore", category=RuntimeWarning) 
simplefilter("ignore", category=UserWarning)

/home/rucinskij/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


## Data Load ##

In [3]:
RANDOM_STATE = 42

x_train_path = "./../../data/x_train.txt"
y_train_path = "./../../data/y_train.txt"

x_test_path = "./../../data/x_test.txt"


X = pd.read_csv(x_train_path, delim_whitespace=True, header=None)
y = pd.read_csv(y_train_path, delim_whitespace=True, header=None)
X_real_test = pd.read_csv(x_test_path, delim_whitespace=True, header=None)


/tmp/ipykernel_1556/1583141569.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X = pd.read_csv(x_train_path, delim_whitespace=True, header=None)
/tmp/ipykernel_1556/1583141569.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(y_train_path, delim_whitespace=True, header=None)
/tmp/ipykernel_1556/1583141569.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_real_test = pd.read_csv(x_test_path, delim_whitespace=True, header=None)


## Grid search ##

In [4]:
cv = 5

model_grid_params = [
    {
        "model": XGBClassifier,
        "params": [{"n_estimators": i, "max_depth": j, "learning_rate": 0.1} for (i,j) in 
                   [(100, 3), (200, 3), (300, 3), (100, 6), (200, 6), (300, 6),(100, 9), (200, 9), (300, 9)]]
                
    },  
]
feature_selector_grid_params = [
    {"selector": SelectFromModel,
    "params": [{"estimator": i, "max_features": max_features} for i in [ExtraTreesClassifier()] for max_features in [1,3,5,7,9]],#LogisticRegression(penalty="l1", solver="saga"),
    },
]


In [5]:
results_dict = run_grid_search_cv(feature_selector_grid_params, model_grid_params, X, y, profit_scoring)

Running on: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...) and SelectFromModel(estimator=ExtraTreesClassifier(), max_features=1)
scores: [1290, 1180, 1270, 1270, 1290]
features: [[102], [103], [102], [105], [100]]
Running on: XGBClassifier(base_score=None, booster=None, callbacks=None,
       

In [6]:
results_dict

{'SelectFromModel(estimator=ExtraTreesClassifier(), max_features=1)': {'XGBClassifier(base_score=None, booster=None, callbacks=None,\n              colsample_bylevel=None, colsample_bynode=None,\n              colsample_bytree=None, device=None, early_stopping_rounds=None,\n              enable_categorical=False, eval_metric=None, feature_types=None,\n              gamma=None, grow_policy=None, importance_type=None,\n              interaction_constraints=None, learning_rate=0.1, max_bin=None,\n              max_cat_threshold=None, max_cat_to_onehot=None,\n              max_delta_step=None, max_depth=3, max_leaves=None,\n              min_child_weight=None, missing=nan, monotone_constraints=None,\n              multi_strategy=None, n_estimators=100, n_jobs=None,\n              num_parallel_tree=None, random_state=None, ...)': {'scores': [1290,
    1180,
    1270,
    1270,
    1290],
   'features': [[102], [103], [102], [105], [100]]},
  'XGBClassifier(base_score=None, booster=None, cal

In [9]:
avg_results_dict = apply_transform_to_res(results_dict, np.mean)
avg_df = pd.DataFrame(avg_results_dict).T
avg_df.columns = [
    "XGBoost:n=100:max_feat=3","XGBoost:n=200:max_feat=3", "XGBoost:n=300:max_feat=3",
    "XGBoost:n=100:max_feat=6","XGBoost:n=200:max_feat=6","XGBoost:n=300:max_feat=6",
    "XGBoost:n=100:max_feat=9","XGBoost:n=200:max_feat=9","XGBoost:n=300:max_feat=9"]
avg_df.style.background_gradient(cmap="Greens", low=0, high=1, axis=None)

,XGBoost:n=100:max_feat=3,XGBoost:n=200:max_feat=3,XGBoost:n=300:max_feat=3,XGBoost:n=100:max_feat=6,XGBoost:n=200:max_feat=6,XGBoost:n=300:max_feat=6,XGBoost:n=100:max_feat=9,XGBoost:n=200:max_feat=9,XGBoost:n=300:max_feat=9
"SelectFromModel(estimator=ExtraTreesClassifier(), max_features=1)",1260.000000,1230.000000,1160.000000,1224.000000,1218.000000,1186.000000,1182.000000,1186.000000,1196.000000
"SelectFromModel(estimator=ExtraTreesClassifier(), max_features=3)",1346.000000,1286.000000,1272.000000,1280.000000,1276.000000,1252.000000,1246.000000,1202.000000,1212.000000
"SelectFromModel(estimator=ExtraTreesClassifier(), max_features=5)",1298.000000,1310.000000,1322.000000,1258.000000,1252.000000,1230.000000,1230.000000,1242.000000,1244.000000
"SelectFromModel(estimator=ExtraTreesClassifier(), max_features=7)",1242.000000,1236.000000,1222.000000,1190.000000,1204.000000,1198.000000,1178.000000,1174.000000,1174.000000
"SelectFromModel(estimator=ExtraTreesClassifier(), max_features=9)",1142.000000,1160.000000,1158.000000,1120.000000,1124.000000,1128.000000,1104.000000,1102.000000,1122.000000


## First dumb try ##

In [7]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [8]:

selected_features = []
max_profit = 0

for feature in X_train.columns:
    selected_features.append(feature)
    
    model = RandomForestClassifier(random_state=RANDOM_STATE)
    model.fit(X_train[selected_features], y_train)
    
    y_pred = model.predict(X_test[selected_features])
    y_pred = y_pred%2
    correct_predictions = ((y_pred == 1) & list((y_test == 1)[0])).sum()
    profit = correct_predictions * 10 - len(selected_features) * 200
    
    if profit > max_profit:
        max_profit = profit

    else:
        selected_features.remove(feature)

print(selected_features)

[0]


In [9]:
max_profit

2610

In [10]:
y_test.sum()*10

0    5080
dtype: int64